## Introduction

* missing values를 처리하는 방법
* missing value가 생기는 경우
    * 2개의 침실을 가진 집의 세번째 침실 사이즈
    * 설문 응답자가 자신의 소득을 공유하지 않았을 때
    
<br>

## Three Approaches

#### 1. A Simple Option: Drop columns with Missing Values
* missing value가 있는 컬럼을 제거하는 가장 간단한 옵션

<img src="../images/3/ASimpleOption.png" width="70%">

* 제거된 컬럼에 missing value인 데이터가 적은 경우, 모델은 유용할 수도 있는 많은 정보를 잃게 됨
    * missing value가 하나라도 있으면 컬럼 전체를 제거하기 때문

#### 2. A Better Option: Imputation
* missing value를 임의의 수로 채움
* 임의의 수는 대부분 정확한(잘 맞는) 값이 아님 but 보통은 컬럼을 제거하는 것보다 좋은 성능을 끌어냄

#### 3. An Extension To Imputation
* 임의의 수는 실제 값(데이터에서 수집되지 않은 값)보다 높거나 낮을 수 있음
* 앞의 imputation처럼 missing value에 임의의 값을 채우는 것은 동일함
* 거기에 추가적으로, imputation이 수행되었다는 것을 알려주는 컬럼을 원래 데이터셋에 추가함
* 이 기법은 경우에 따라 성능을 향상시키기도 하고, 전혀 영향을 주지 못하기도 함

<img src="../images/3/AnExtensionToImputation.png" width="70%">

## Example

* 집 값을 예측하는 데 '방의 개수'와 같은 정보를 사용

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

## Load the data ##
data = pd.read_csv('../input/melb_data.csv')

## select target ##
y = data.Price

## To keep things simple, we'll use only numerical predictors ##
melb_predictors = data.drop(['Price'], axis=1) # Price 컬럼 제거
X = melb_predictors.select_dtypes(exclude=['object']) # object 타입 제외

## Divide data into training and validation subsets ##
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

* drop(labels=None, axis=0, ...): 지정된 axis에서 label에 해당하는 column이나 row를 제거
    * labels: 삭제하고자 하는 데이터 이름
    * axis: labels가 있는 axis
* select_dtypes(include=None, exclude=None): 컬럼의 데이터 타입을 기반으로 DaraFrame의 컬럼 subset을 반환
    * include: subset에 포함할 컬럼 
    * exclude: subset에서 제외할 컬럼

<br>

#### Define Function to Measure Quality of Each Approach

* missing values를 처리하는 여러 기법의 성능을 비교하기 위한 함수(score_dataset()) 정의

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

## Function for comparing different approaches ##
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0) # n_estimators: randomforest의 tree 개수
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    
    return mean_absolute_error(y_valid, preds)

#### Score from Approach 1 (Drop Columns with Missing Values)

In [3]:
## Get names of columns with missing values ##
cols_with_missing = [col for col in X_train.columns
                        if X_train[col].isnull().any()] # any(): iterable 객체의 요소 중 어느 하나라도 True면 True 반환

## Drop columns in training and validation data ##
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop colums with missing values): ")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop colums with missing values): 
183550.22137772635


#### Score from Approach 2 (Imputation)

* missing values를 해당 컬럼의 mean 값으로 채움
* 간단한 방법이지만 일반적으로 꽤 좋은 성능을 냄
* 반면에 통계학자들은 임의의 수를 훨씬 복잡하게 결정(ex. regression imputation)
    * 복잡한 머신 러닝 모델에서는 일반적으로 장점이 없음

In [4]:
from sklearn.impute import SimpleImputer

## Imputation ##
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.fit_transform(X_valid))

## Imputation removed column names; put them back ##
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation): ")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation): 
179816.89508731329


* SimpleImputer(missing_values=nan, strategy='mean', fill_value=None, ...)
    * missing_values: number, string, nan, none. 지정한 값에 해당하는 데이터에 imputation 수행
    * strategy: mean, median, most_frequent, constant
    * fill_value: strategy가 constant일 때, 이 값으로 missing_value를 채움

#### Score from Approach 3 (An Extension to Imputation)

In [5]:
## Make copy to avoid changing original data (when imputing) ##
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

## Make new columns indicating what will be imputed ##
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()
    
## Imputation ##
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.fit_transform(X_valid_plus))

## Imputation removed column names; put them back ##
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation): ")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation): 
179986.2708570026


* Approach 3이 Approach 2보다 성능이 약간 낮음

<br>

#### So, why did imputation perform better than dropping the columns?

* 학습 데이터셋은 10864개의 row와 12개의 column, 3개의 missing column으로 이루어짐
* 각 column의 missing value 개수는 전체 데이터의 절반 이하
    * column을 전부 제거하는 것은 수 많은 유용한 정보를 잃는 것임

In [6]:
## Shape of training data(num_rows, num_columns) ##
print(X_train.shape)

## Number of missing values in each column of training data ##
missing_val_count_by_column = (X_train.isnull().sum())
# print(missing_val_count_by_column)
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
